In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4
%pip install -qU langchain-openai
!pip gradio_client==0.2.10
!pip install gradio==3.38.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-experimental 0.0.61 requires langchain-community<0.3.0,>=0.2.5, but you have langchain-community 0.3.21 which is incompatible.
langchain-experimental 0.0.61 requires langchain-core<0.3.0,>=0.2.7, but you have langchain-core 0.3.52 which is incompatible.
langchain-openai 0.1.17 requires langchain-core<0.3.0,>=0.2.20, but you have langchain-core 0.3.52 which is incompatible.
streamlit 1.32.2 requires protobuf<5,>=3.20, but you have protobuf 5.29.4 which is incompatible.
spyder 5.5.1 requires ipython!=8.17.1,<9.0.0,>=8.13.0; python_version > "3.8", but you have ipython 9.1.0 which is incompatible.
ERROR: unknown command "gradio_client==0.2.10"

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency c

In [4]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [8]:
from langchain_openai import ChatOpenAI
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [9]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [ ]:
#from langchain_community.document_loaders import WebBaseLoader
#url = 'https://aws.amazon.com/architecture/well-architected'
#loader = WebBaseLoader(url)
#docs = loader.load()
#docs

In [10]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

In [12]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings(model='text-embedding-3-small'))
retriever = vectorstore.as_retriever()

#print(docs[0].page_content)
#len(docs[0].page_content.split())
#docs
#len(docs)

In [13]:
# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
#   "You are an assistant for question-answering tasks. "
#     "Use the following pieces of retrieved context to answer "
#     "the question. If you don't know the answer, say that you "
#     "don't know. Use three sentences maximum and keep the "
#     "answer concise."
#     "\n\n"

#     "LangChain is a framework that does ...."
#     "What should I used to productionize langchain"

In [14]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)
# retriever | prompt | llm

In [15]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
print("Response is\n------")
print(response["answer"])
# print(response["context"])

Response is
------
Task decomposition involves breaking down a complex task into smaller and simpler steps. It helps in transforming a big task into multiple manageable tasks, enabling a model or agent to utilize more computation at test time. Methods like Chain of Thought (CoT) prompt the model to think step by step, aiding in the interpretation of the model's thinking process.


In [16]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
# If there is no chat_history, then the input is just passed directly to the
# retriever. If there is chat_history, then the prompt and LLM will be used to
# generate a search query. That search query is then passed to the retriever.

# This chain prepends a rephrasing of the input query to our retriever,
# so that the retrieval incorporates the context of the conversation.

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [17]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# history_aware_retriever and question_answer_chain in sequence, retaining
# intermediate outputs such as the retrieved context for convenience.
# It has input keys input and chat_history, and includes input, chat_history,
# context, and answer in its output.
rag_chain = create_retrieval_chain(history_aware_retriever,
                                   question_answer_chain)
#  contextualize_q_prompt | llm1 | retriver | qa_prpmt | llm2

In [18]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are common ways of doing it?" # What are common ways of doing task decompsition
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Task decomposition can be achieved in various ways, including using techniques like Chain of Thought (CoT) to prompt models to break down tasks step by step. Additionally, it can be done by providing simple instructions or subgoals to guide the decomposition process, utilizing task-specific prompts, or incorporating human inputs for guidance. Overall, task decomposition strategies aim to simplify complex tasks by breaking them down into smaller, more manageable components.


In [19]:
import gradio as gr

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [20]:
from langchain_core.messages import AIMessage, HumanMessage

def predict(question, chat_history):
  history_for_llm = []
  for tup in chat_history:
    history_for_llm.extend([HumanMessage(tup[0]), AIMessage(tup[1])])

  ai_msg = rag_chain.invoke({"input": question, "chat_history": history_for_llm})
  return ai_msg["answer"]

In [21]:
demo = gr.ChatInterface(predict,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Hi I am your virtual assistant, how can I help you today?", container=False, scale=7),
    title="DocumentQABot",
    theme="soft",
    examples=["What is the weather like in SF?", "What is LangSmith?"],
    retry_btn=None,
    undo_btn="Delete Previous",
    clear_btn="Clear",)
demo.launch(share=True, debug=True)

NameError: name 'gr' is not defined